# Machine Learning & XAI
Wir möchten nun herausfinden, wie die verschiedenen Modelle mit den Daten perfomen und welche Unterschiede wir entdecken können.

Hinweis zu den Quellen:
In den letzten Semestern an der ZHAW haben wir uns in verschiedenen Modulen wie z.B. Data Management, Data Science Introduction, Scientific Programming, Machine Learning I, Machine Learning II und Model Deployment & Maintenance mit dem Thema Large Languages Models beschäftigt. Je nach Modul eher theoretisch mit Statisik und Konzepten oder eher praktisch mit Python, Keras, Tensorflow, Huggingface, etc. in entsprechenden Projekten. Für dieses Notebook habe ich entsprechenden Ansätze und Code bei Bedarf verwendet, dabei habe ich die entsprechenden Module als Quelle angegeben, egal ob ich "nur" das theoretische Konzept oder auch Teile des Codes verwendet habe.

## Import von benötigten Libraries & dem Dataset

Diese Libraries werden für die Bearbeitung des vorliegenden Notebooks verwendet. Bitte stellen Sie sicher, dass deren Installation mithilfe von `pip install <library>` sichergestellt ist.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk 
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline


# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

pd.set_option('display.max_colwidth', 100)

import re,string,unicodedata
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
from tensorflow.keras.optimizers import Adam

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.pipeline import Pipeline

# Bewertung von Modelen
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

#XAI Anwendungen
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Attention
from tensorflow.keras.models import Model

c:\Users\olive\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Dataset
Als ersten Schritt importiere ich das Datenset des vorhergebenen Schrittes 06_Feature Selection & Engineering

Quellen:
- Code validiert mit Chat-GPT 3.5

In [25]:
cleaned_data_df = pd.read_csv('..\\06_Feature Selection & Engineering\\06_dataset_tweets_featured.csv')

## Fine Tuning

Der Datensatz ist zu klein um ein ganze Large Language Model zu trainieren. Daher verwende ich das bereits trainierte Model "bert-base-uncased" von Huggingface inkl. entsprechendem Tokenizer dazu um das Model für den vorliegenden Datensatz Fine zu tunen. 

Das Bert-Model habe ich bereits im Modul "Model Deployment & Maintenance" im letzten Semester verwendet, daher habe ich entschieden es hier wieder zu verwenden.

Quellen:
- Model Deployment & Maintenance, ZHAW, FS2023
- huggingface.co
- Code validiert mit Chat-GPT 3.5

In [4]:
model_name = 'bert-base-uncased'

max_length = 140
config = BertConfig.from_pretrained(model_name)
#config.output_hidden_states = False

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
bert = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Für das weitere Vorgehen mit dem Model erstelle ich zwei Eingabeschichten (input_ids und attention_mask), um die Tweets zu verarbeiten. Das BERT-Modell wird dann auf diese Eingabeschichten angewendet, und der Ausgang wird in der Variable x gespeichert.

Quellen:
- Model Deployment & Maintenance, ZHAW, FS2023
- Code validiert mit Chat-GPT 3.5

In [5]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
x = bert.bert(inputs)

Für das Training des Models verwende ich die bereinigten Tweet-Texte und Klassifizierung in der Spalte cyberbulling_type. Daher kann ich die restlichen Spalten aus dem Dataframe entfernen. Da ich ein Pretrained Model verwende, dass Kleinschrebung ausgelegt ist, wandle ich auch alle Tweet-Texte in Kleinbuchstaben um.

Quellen:
- Code validiert mit Chat-GPT 3.5

In [27]:
cleaned_data_df = cleaned_data_df.drop(columns=['interaction_id', 'tweet_id', 'senderuser_id', 'receiveruser_id', 'interaction_timestamp', 'char_count', 'harass', 'cyberstalk', 'doxing', 'exclusion', 'impersonation', 'trolling', 'hateSpeech', 'threats', 'flaming', 'outing', 'word_count', 'avg_word_length', 'avg_sentence_length', 'punctuation_count'])
cleaned_data_df['tweet_text'] = cleaned_data_df['tweet_text'].str.lower()
cleaned_data_df

,cyberbullying_type,tweet_text
0,not_cyberbullying,@GYU WHAT IS THAT
1,other_cyberbullying,@STOCKPUTOUT YES
2,other_cyberbullying,
3,none,@BLACKAMAZON EXACTLY THIS
4,none,@ODDTANKOUT HTTP T CO KMMJEUME
...,...,...
64339,ethnicity,ONLY THING THAT HEATS MY HEAD MORE IS WHEN I SEE A BLACK PEOPLE ACTING LIKE NIGGERS AND B ME...
64340,age,FOUND OUT THE GIRL WHO SPREAD (TRUE) RUMOURS ABOUT ME BEING BI AND KINDA BULLIED ME ABOUT IT IN ...
64341,not_cyberbullying,NIÑAS REINAS DEL “BULLYING” EN ESCUELAS PRIVADAS HTTP T CO VKAZXI Y LAS Q SE CREEN QUEEN B...
64342,religion,@DIANH AS USUAL YOU ARE A MUSLIM LIAR MAKING ANY EXCUSES FOR A RELIGION OF BARBARITY MURDER H...


Damit das Finetuning auch nur mit Zeilen mit effektivem Inhalt der Spalte tweet_text geschieht und es keine Verzerrungen gibt, stelle ich nochmals sicher, dass die Zeilen mit leerem Inhalt in der entsprechenden Zelle in der Spalte tweet_text komplett entfernt werden.

In [29]:
cleaned_data_df = cleaned_data_df.dropna(subset=['tweet_text'])

Da es sich hier um mehrere Klassen handelt und ich damit das Model trainieren möchte nutze ich die Funktion "get_dumies" um sie für das Machine Learning zu optimieren. Die Funktion get_dummies in Pandas wird verwendet, um Dummy-Variablen für kategorische Spalten zu erstellen, wodurch diese Spalten in eine binäre (0 oder 1) Darstellung umgewandelt werden.

Quellen:
- Machine Learning I, ZHAW, HS2022
- Code validiert mit Chat-GPT 3.5

In [30]:
df_la = pd.get_dummies(cleaned_data_df, columns = ['cyberbullying_type'])
df_la

,tweet_text,cyberbullying_type_age,cyberbullying_type_ethnicity,cyberbullying_type_gender,cyberbullying_type_none,cyberbullying_type_not_cyberbullying,cyberbullying_type_other_cyberbullying,cyberbullying_type_racism,cyberbullying_type_religion,cyberbullying_type_sexism
0,@GYU WHAT IS THAT,0,0,0,0,1,0,0,0,0
1,@STOCKPUTOUT YES,0,0,0,0,0,1,0,0,0
2,,0,0,0,0,0,1,0,0,0
3,@BLACKAMAZON EXACTLY THIS,0,0,0,1,0,0,0,0,0
4,@ODDTANKOUT HTTP T CO KMMJEUME,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
64339,ONLY THING THAT HEATS MY HEAD MORE IS WHEN I SEE A BLACK PEOPLE ACTING LIKE NIGGERS AND B ME...,0,1,0,0,0,0,0,0,0
64340,FOUND OUT THE GIRL WHO SPREAD (TRUE) RUMOURS ABOUT ME BEING BI AND KINDA BULLIED ME ABOUT IT IN ...,1,0,0,0,0,0,0,0,0
64341,NIÑAS REINAS DEL “BULLYING” EN ESCUELAS PRIVADAS HTTP T CO VKAZXI Y LAS Q SE CREEN QUEEN B...,0,0,0,0,1,0,0,0,0
64342,@DIANH AS USUAL YOU ARE A MUSLIM LIAR MAKING ANY EXCUSES FOR A RELIGION OF BARBARITY MURDER H...,0,0,0,0,0,0,0,1,0


Hier werden nun die tweet_texte in train_sentences übertragen und die Klassen für das Finetuning vorbereitet. Insgesamt wird dieses Setup oft in multiklassigen Klassifikationsaufgaben verwendet, bei denen für jeden Text mehrere Klassenlabels vorhergesagt werden sollen. Das Modell wird dann darauf trainiert, Muster in den Texten zu lernen und die relevanten Klassen für jeden Text vorherzusagen.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Code validiert mit Chat-GPT 3.5

In [31]:
train_sentences = cleaned_data_df["tweet_text"].values
list_classes = ['cyberbullying_type_age','cyberbullying_type_ethnicity','cyberbullying_type_gender','cyberbullying_type_not_cyberbullying','cyberbullying_type_other_cyberbullying','cyberbullying_type_religion']
train_y = df_la[list_classes].values

Hier sollen nun die Textdaten verarbeitet werden wobei daraus relevante Merkmale extrahiert werden sollen und dann versucht wird, auf dessen Basis Wahrscheinlichkeiten für verschiedene Klassen vorherzusagen.

Quellen:
- Model Deployment & Maintenance, ZHAW, FS2023
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [32]:
#x2 =Dense(512, activation='relu')(x[1])
x2 = GlobalAveragePooling1D()(x[0])
#x3 = Dropout(0.5)(x2)
y =Dense(len(list_classes), activation='sigmoid', name='outputs')(x2)

model = Model(inputs=inputs, outputs=y)
#model.layers[2].trainable = False

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

Der Adam-Optimizer ist eine Variante des stochastischen Gradientenabstiegs (SGD), der häufig für das Training von neuronalen Netzwerken verwendet wird. Die Lernrate steuert die Größe der Schritte, die der Optimierer während des Trainings macht, und 1e-5 ist eine typische Anfangs-Lernrate. Dieser Code das neuronale Netzwerk darauf vor, auf die gegebene Aufgabe mit einem bestimmten Optimierer und einer spezifischen Lernrate trainiert zu werden.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [33]:
optimizer = Adam(learning_rate=1e-5)

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)

Die Inputs werden als Token an das BERT-Modell übergeben, damit das Model die Inhalte der Tweets verarbeiten kann. Die Ausgabe des Codes ist ein Dictionary von TensorFlow-Tensoren, das die tokenisierten Sequenzen ('input_ids') enthält.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [34]:
# Tokenize the input 
x = tokenizer(
    text=list(train_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

Mit vorbereiteten Daten kann nun das Model trainiert werden. Dazu verwende ich die Fit-Funktion. Das ist nun der verarbeitende Schritt. Bitte beachten Sie dass das Training je nach Hardware einige Zeit in Anspruch nehmen kann. Auf meinem PC hat das Training mit einem Epochs ca. 10 Stunden gedauert und eine Accuracy von ca. 0.58 erreicht. Daher empfiehlt es sich das Model mit ca. 3 Epochs zu trainieren und anschliessend zu speichern.

Quellen:
- Model Deployment & Maintenance, ZHAW, FS2023
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [35]:
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    #x={'input_ids': x['input_ids']},
    y={'outputs': train_y},
    validation_split=0.5,
    batch_size=32,
    epochs=1)

1006/1006 [==============================] - 42531s 42s/step - loss: 0.1510 - accuracy: 0.5898 - val_loss: 0.1236 - val_accuracy: 0.6251


In [ ]:
model.save("07_Anwendung in Machine Learning/Finetuning_BERT_Twitter")

# Laden des gespeicherten Modells
loaded_model = keras.models.load_model(
    "07_Anwendung in Machine Learning/Finetuning_BERT_Twitter"
)

# Jetzt kannst du das geladene Modell für Vorhersagen verwenden
# Zum Beispiel: loaded_model.predict(input_data)

Für die Überprüfung des Models splitte ich den Datensatz in einen Trainings- und Testdatensatz.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data_df['tweet_text'], all_data_df['cyberbullying_type'],
                                                    test_size = 0.2, random_state = 42)
print(f'Data Split done.')

Der Vectorizer wird nun auf die Daten angepasst, um das Vokabular zu erstellen und die TF-IDF-Gewichtungen zu berechnen.

Quellen:
- Code opimiert und validiert mit Chat-GPT 3.5

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=500000)
vectorizer.fit(X_train)
print(f'Vectorizer fitted.')
print('No. of feature_words: ', len(vectorizer.get_feature_names_out()))

In [ ]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)
print(f'Data Transformed.')

Die Funktion model_Evaluate führt eine Evaluierung eines Modells durch und gibt verschiedene Leistungsmetriken aus. Dies nutze ich als Basis bzw. Ausgangslage für Evaluierung mit den weiteren Modellen.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Code opimiert und validiert mit Chat-GPT 3.5

In [ ]:
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    
    cm=confusion_matrix(y_pred , y_test)
    plt.figure()
    plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
    plt.xticks(range(2), ['Negative',  'Positive'], fontsize=16,color='black')
    plt.yticks(range(2), ['Negative', 'Positive'], fontsize=16)
    plt.show()

## Modelbewertung

### Decision Tree Classifier
Der Decision Tree Classifier ist ein Supervised Machine Learning Algorithmus, der für Klassifikationsaufgaben verwendet wird. Seine Stärke liegt in seiner Fähigkeit, komplexe Entscheidungsregeln auf Basis der Trainingsdaten zu erlernen und präzise Vorhersagen zu treffen. Die Entscheidungsregeln werden in Form eines Baumes dargestellt, wobei jeder innere Knoten eine Entscheidung darstellt und jeder Blattknoten ein Ergebnis darstellt.

In [ ]:
dtc= DecisionTreeClassifier()
dtc.fit(X_train, y_train)
model_Evaluate(dtc)

Da es hier um ein Model mit mehreren Klassen handelt, nutze die Methode "OneVSOneClassifier" um die Klassen zu klassifizieren. Die Methode "OneVSOneClassifier" ist eine Methode, die eine Reihe von binären Klassifikatoren erstellt, die jeweils zwei Klassen unterscheiden. Die Entscheidung wird dann durch Abstimmung getroffen.

Quellen:
- Machine Learning II, ZHAW, FS2023
- Chat-GPT 3.5
- Code optimiert mit Github Co-Pilot

In [ ]:
y_score = label_binarize(dtc.predict(X_test), classes=dtc.classes_)
print(y_score)

# Binarisieren Sie die multiklassen Labels für die Testdaten
y_test_bin = label_binarize(y_test, classes=dtc.classes_)
print(y_test_bin)

# Berechnung der ROC-Kurve für jede Klasse
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)


plt.figure(figsize=(7, 7))
plt.plot(fpr_micro, tpr_micro, color='darkred', lw=2, label='Micro-average ROC curve (area = {:0.2f})'.format(roc_auc_micro))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree Classifier (Micro-average)')
plt.legend(loc='lower right', fontsize=13)
plt.show()

### Logistic Regression
Die logistische Regression ist einfach, interpretierbar und effektiv für binäre Klassifikationsaufgaben. Wenn es um mehr als zwei Klassen geht, kann die logistische Regression auf verschiedene Arten erweitert werden, wie z. B. die "One-vs-Rest" oder "Multinomial" Ansätze.

Quellen:
- Machine Learning I, ZHAW, HS2022
- Chat-GPT 3.5
- Code optimiert mit Github Co-Pilot

In [ ]:
lr=LogisticRegression()
lr.fit(X_train, y_train)
model_Evaluate(lr)

In [ ]:
y_score = lr.predict_proba(X_test)
y_test_bin = label_binarize(y_test, classes=lr.classes_)

fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()
for i in range(len(lr.classes_)):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])


plt.figure(figsize=(7, 7))
for i in range(len(lr.classes_)):
    plt.plot(
        fpr_macro[i],
        tpr_macro[i],
        lw=2,
        label="ROC curve (area = {:0.2f}) for class {}".format(roc_auc_macro[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Logistic Regression (Macro-average)")
plt.legend(loc="lower right", fontsize=13)
plt.show()

### KNeighbors Classifier
Der K-Nearest Neighbors Classifier ist ein einfacher und intuitiver Algorithmus für maschinelles Lernen, der für Klassifikations- und Regressionsaufgaben verwendet wird. Der KNN-Classifier ist besonders nützlich in Situationen, in denen die Entscheidungsgrenze nicht linear ist und die Datenpunkte nah beieinander gruppiert sind. Es ist wichtig, die Anzahl der Nachbarn und die Entfernungsmaße sorgfältig zu wählen, um optimale Ergebnisse zu erzielen.
Diesen Ansatz habe ich gewählt, da ich die Vermutung habe, dass die Datenpunkte der verschiedenen Klassen nahe beieinander gruppiert sein könnten.

Quellen:
- Chat-GPT 3.5
- Code optimiert mit Github Co-Pilot

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
model_Evaluate(knn)

In [ ]:
y_score = knn.predict_proba(X_test)
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()

for i in range(n_classes):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])

    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])

plt.figure(figsize=(7, 7))
for i in range(n_classes):

    plt.plot(
        fpr_macro[i],
        tpr_macro[i],
        lw=2,
        label="ROC curve (area = {:0.2f}) for class {}".format(roc_auc_macro[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("K-Nearest Neighbors (Macro-average)")
plt.legend(loc="lower right", fontsize=13)

plt.show()

### AdaBoost Classifier
AdaBoost ist ein sogenannter Boosting-Algorithmus, der auf dem Prinzip des Schichtens von Klassifikatoren basiert.
Er verwendet eine Reihe von schwachen Klassifikatoren und kombiniert ihre Vorhersagen, um eine genauere Gesamtvorhersage zu erzielen.

Quellen:
- Chat-GPT 3.5
- Code optimiert mit Github Co-Pilot

In [ ]:
abc = AdaBoostClassifier()
abc.fit(X_train,y_train)
model_Evaluate(abc)

In [ ]:
y_score = abc.predict_proba(X_test)
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()

for i in range(n_classes):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])

plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(
        fpr_macro[i],
        tpr_macro[i],
        lw=2,
        label="ROC curve (area = {:0.2f}) for class {}".format(roc_auc_macro[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AdaBoost Classifier (Macro-average)")
plt.legend(loc="lower right", fontsize=13)
plt.show()

### MultiNomial NB
Multinomial Naive Bayes (MNB) ist ein probabilistischer Klassifikationsalgorithmus, der auf dem Bayes-Theorem basiert und insbesondere für Textklassifikationsprobleme geeignet ist. Multinomial Naive Bayes ist besonders nützlich für Textklassifikationsaufgaben. Daher habe ich diesen Ansatz gewählt um das vorliegende Modell ebenfalls mit diesem Ansatz zu überprüfen.

Quellen:
- Chat-GPT 3.5
- Code optimiert mit Github Co-Pilot

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
model_Evaluate(mnb)

In [ ]:
y_score = mnb.predict_proba(X_test)
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

f, axes = plt.subplots(1, 1, figsize=(7, 7))
axes.plot(
    fpr_micro,
    tpr_micro,
    color="darkred",
    lw=2,
    label="Micro-average ROC curve (area = {:0.2f})".format(roc_auc_micro),
)
axes.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="Multi-Nomial Naive Bayes (Micro-average)",
)
axes.legend(loc="lower right", fontsize=13)
plt.show()

### Gradient Boosting Classifier
Gradient Boosting ist eine Ensemble-Methode, die darauf abzielt, schwache Elemente zu einem starken Element zu kombinieren. Der Gradient Boosting Classifier ist speziell für Klassifikationsaufgaben ausgelegt und da es sich hier um eine Klassifikationsaufgabe handelt, habe ich diesen Ansatz gewählt um das vorliegende Modell ebenfalls mit diesem Ansatz zu überprüfen.

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
model_Evaluate(gbc)

In [ ]:
model = OneVsRestClassifier(GradientBoostingClassifier())
y_score = model.fit(X_train, y_train).decision_function(X_test)
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(
        fpr[i],
        tpr[i],
        lw=2,
        label="ROC curve (area = {:0.2f}) for class {}".format(roc_auc[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Gradient Boosting Classifier (One-vs-Rest ROC)")
plt.legend(loc="lower right", fontsize=13)
plt.show()

### Random Forest Clssifier
Der Random Forest Classifier ist eine Methode, die aus einer Sammlung von Entscheidungsbäumen besteht. Diese Bäume werden unabhängig voneinander trainiert, und ihre Vorhersagen werden kombiniert, um die Gesamtvorhersage zu verbessern. Der Random Forest Classifier ist speziell für Klassifikationsaufgaben ausgelegt, daher habe ich es ebenfalls gewählt um das vorliegende Modell ebenfalls mit diesem Ansatz zu überprüfen.

In [ ]:
rfc = RFC(random_state=42)
rfc.fit(X_train, y_train)
model_Evaluate(rfc)

In [ ]:
pred_rfc = rfc.predict_proba(X_test)[:,1]
fpr_rfc,tpr_rfc,_ = roc_curve(y_test.values,pred_rfc)
roc_auc_rfc = auc(fpr_rfc,tpr_rfc)


f, axes = plt.subplots(1, 1,figsize=(7,7))
axes.plot(fpr_rfc, tpr_rfc, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_rfc))
axes.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Random Forest Classifer')
axes.legend(loc='lower right', fontsize=13)

# XAI Modelanalyse
XAI steht für Explainable Artificial Intelligence. Diese Technik konzentriert sich darauf, komplexe KI-Modelle und deren Entscheidungen transparenter und nachvollziehbarer zu machen. Das Ziel von XAI ist es, das "Black-Box"-Problem anzugehen, bei dem tiefgreifende Modelle, insbesondere neuronale Netzwerke, als schwer interpretierbar gelten. (Chat-GPT 3.5, 2023)

In [ ]:
input_layer = Input(shape=(max_length,), dtype="int32")
embedding_layer = Embedding(input_dim=500000, output_dim=128, input_length=max_length)(
    input_layer
)

# Define the LSTM layer
lstm_layer = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# Define the output layer
output_layer = Dense(1, activation="sigmoid")(lstm_layer)

# Define the model architecture
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

# Print the model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.2
)

In [ ]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)

# Print test accuracy
print("\n", "Test accuracy:", score[1])

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 5), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, 5), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 5), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, 5), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.show()

# Annahme: model ist Ihr LSTM-Modell
y_score = model.predict(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Berechnung der Mikro-Durchschnitt-ROC-Kurve
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Plotting
f, axes = plt.subplots(1, 1, figsize=(7, 7))
axes.plot(
    fpr_micro,
    tpr_micro,
    color="darkred",
    lw=2,
    label="Micro-average ROC curve (area = {:0.2f})".format(roc_auc_micro),
)
axes.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="LSTM (Micro-average)",
)
axes.legend(loc="lower right", fontsize=13)
plt.show()

In [ ]:
model = OneVsRestClassifier(model)
y_score = model.fit(X_train, y_train).decision_function(X_test)

n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(
        fpr[i],
        tpr[i],
        lw=2,
        label="ROC curve (area = {:0.2f}) for class {}".format(roc_auc[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("LSTM (One-vs-Rest ROC)")
plt.legend(loc="lower right", fontsize=13)
plt.show()